In [5]:
from etl.spark.spark_session_helper import spark

23/02/18 22:01:44 WARN Utils: Your hostname, Chaturvedi_PC resolves to a loopback address: 127.0.1.1; using 172.28.53.96 instead (on interface eth0)
23/02/18 22:01:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/18 22:01:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
from path_manager import intermediate_data_all_train_rows_path

/home/abhay/work/dream11


In [7]:
all_player_rows_df = spark.read.parquet(intermediate_data_all_train_rows_path)
all_player_rows_df.registerTempTable("all_player_rows")

23/02/18 22:01:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
spark.sql("""
    select distinct(venue_name) from all_player_rows where 
    dt>date_sub(current_timestamp(), 1000)
    and lower(venue_name) like '%george%'
""").limit(30).toPandas()

,venue_name
0,"St George's Park, Port Elizabeth"
1,"National Cricket Stadium, St George's, Grenada"
2,"St George's Park, Gqeberha"


In [27]:
from string import Template
feature_query_template = Template("""
select 
    player_id, '$venue_name' as venue_name,
    
    is_male,
    sum(case when dt>date_sub(current_timestamp(), 1000) then batter_run_sum else 0 end) as batter_runs_1000D,
    sum(case when dt>date_sub(current_timestamp(), 1000) and venue_name='$venue_name' then batter_run_sum else 0 end) as batter_runs_1000D_venue,
    sum(case when dt>date_sub(current_timestamp(), 1000) then balls_faced else 0 end) as balls_faced_1000D,
    sum(case when dt>date_sub(current_timestamp(), 1000) and venue_name='$venue_name' then balls_faced else 0 end) as balls_faced_1000D_venue,
    sum(case when dt>date_sub(current_timestamp(), 1000) then dismissals else 0 end) as dismissals_1000D,
    sum(case when dt>date_sub(current_timestamp(), 1000) and venue_name='$venue_name' then dismissals else 0 end) as dismissals_1000D_venue,
    sum(case when dt>date_sub(current_timestamp(), 1000) then boundary_count else 0 end) as boundary_count_1000D,
    sum(case when dt>date_sub(current_timestamp(), 1000) and venue_name='$venue_name' then boundary_count else 0 end) as boundary_count_1000D_venue,
    sum(case when dt>date_sub(current_timestamp(), 1000) then six_count else 0 end) as six_count_1000D,
    sum(case when dt>date_sub(current_timestamp(), 1000) and venue_name='$venue_name' then six_count else 0 end) as six_count_1000D_venue,
    
    sum(case when dt>date_sub(current_timestamp(), 300) then batter_run_sum else 0 end) as batter_runs_300D,
    sum(case when dt>date_sub(current_timestamp(), 300) and venue_name='$venue_name' then batter_run_sum else 0 end) as batter_runs_300D_venue,
    sum(case when dt>date_sub(current_timestamp(), 300) then balls_faced else 0 end) as balls_faced_300D,
    sum(case when dt>date_sub(current_timestamp(), 300) then dismissals else 0 end) as dismissals_300D,
    sum(case when dt>date_sub(current_timestamp(), 300) then boundary_count else 0 end) as boundary_count_300D,
    sum(case when dt>date_sub(current_timestamp(), 300) then six_count else 0 end) as six_count_300D,
    
    sum(case when dt>date_sub(current_timestamp(), 90) then batter_run_sum else 0 end) as batter_runs_90D,
    sum(case when dt>date_sub(current_timestamp(), 90) and venue_name='$venue_name' then batter_run_sum else 0 end) as batter_runs_90D_venue,
    sum(case when dt>date_sub(current_timestamp(), 90) then balls_faced else 0 end) as balls_faced_90D,
    sum(case when dt>date_sub(current_timestamp(), 90) then dismissals else 0 end) as dismissals_90D,
    sum(case when dt>date_sub(current_timestamp(), 90) then boundary_count else 0 end) as boundary_count_90D,
    sum(case when dt>date_sub(current_timestamp(), 90) then six_count else 0 end) as six_count_90D,
    
    sum(case when dt>date_sub(current_timestamp(), 30) then batter_run_sum else 0 end) as batter_runs_30D,
    sum(case when dt>date_sub(current_timestamp(), 30) and venue_name='$venue_name' then batter_run_sum else 0 end) as batter_runs_30D_venue,
    sum(case when dt>date_sub(current_timestamp(), 30) then balls_faced else 0 end) as balls_faced_30D,
    sum(case when dt>date_sub(current_timestamp(), 30) then dismissals else 0 end) as dismissals_30D,
    sum(case when dt>date_sub(current_timestamp(), 30) then boundary_count else 0 end) as boundary_count_30D,
    sum(case when dt>date_sub(current_timestamp(), 30) then six_count else 0 end) as six_count_30D,
    
    sum(case when dt>date_sub(current_timestamp(), 1000) then total_run_sum else 0 end) as total_runs_1000D,
    sum(case when dt>date_sub(current_timestamp(), 1000) and venue_name='$venue_name' then total_run_sum else 0 end) as total_runs_1000D_venue,
    sum(case when dt>date_sub(current_timestamp(), 1000) then deliveries else 0 end) as deliveries_1000D,
    sum(case when dt>date_sub(current_timestamp(), 1000) and venue_name='$venue_name' then deliveries else 0 end) as deliveries_1000D_venue,
    sum(case when dt>date_sub(current_timestamp(), 1000) then wicket_sum else 0 end) as wicket_sum_1000D,
    sum(case when dt>date_sub(current_timestamp(), 1000) and venue_name='$venue_name' then wicket_sum else 0 end) as wicket_sum_1000D_venue,
    sum(case when dt>date_sub(current_timestamp(), 1000) then maiden_count else 0 end) as maiden_count_1000D,
    sum(case when dt>date_sub(current_timestamp(), 1000) and venue_name='$venue_name' then maiden_count else 0 end) as maiden_count_1000D_venue,
    
    sum(case when dt>date_sub(current_timestamp(), 300) then total_run_sum else 0 end) as total_runs_300D,    
    sum(case when dt>date_sub(current_timestamp(), 300) then deliveries else 0 end) as deliveries_300D,
    sum(case when dt>date_sub(current_timestamp(), 300) then wicket_sum else 0 end) as wicket_sum_300D,
    sum(case when dt>date_sub(current_timestamp(), 300) then maiden_count else 0 end) as maiden_count_300D,
    
    sum(case when dt>date_sub(current_timestamp(), 90) then total_run_sum else 0 end) as total_runs_90D,    
    sum(case when dt>date_sub(current_timestamp(), 90) then deliveries else 0 end) as deliveries_90D,
    sum(case when dt>date_sub(current_timestamp(), 90) then wicket_sum else 0 end) as wicket_sum_90D,
    sum(case when dt>date_sub(current_timestamp(), 90) then maiden_count else 0 end) as maiden_count_90D,
    
    sum(case when dt>date_sub(current_timestamp(), 30) then total_run_sum else 0 end) as total_runs_30D,    
    sum(case when dt>date_sub(current_timestamp(), 30) then deliveries else 0 end) as deliveries_30D,
    sum(case when dt>date_sub(current_timestamp(), 30) then wicket_sum else 0 end) as wicket_sum_30D,
    sum(case when dt>date_sub(current_timestamp(), 30) then maiden_count else 0 end) as maiden_count_30D,
    
    
    sum(case when dt>date_sub(current_timestamp(), 1000) then fielding_wicket_sum else 0 end) as fielding_wicket_sum_1000D,
    sum(case when dt>date_sub(current_timestamp(), 30) then fielding_wicket_sum else 0 end) as fielding_wicket_sum_30D
            
    from all_player_rows
    group by player_id, is_male
    having is_male=0
""")
feature_df = spark.sql(feature_query_template.substitute(venue_name="St George\\'s Park, Gqeberha"))

In [28]:
from pyspark.sql import functions as f
feature_df_with_bat_avg = feature_df\
        .withColumn("batting_avg_30D", f.when(f.col("dismissals_30D") > 0, f.col("batter_runs_30D")/f.col("dismissals_30D")).otherwise(f.col("batter_runs_30D"))) \
        .withColumn("batting_avg_90D", f.when(f.col("dismissals_90D") > 0, f.col("batter_runs_90D")/f.col("dismissals_90D")).otherwise(f.col("batter_runs_90D"))) \
        .withColumn("batting_avg_300D", f.when(f.col("dismissals_300D") > 0, f.col("batter_runs_300D")/f.col("dismissals_300D")).otherwise(f.col("batter_runs_300D"))) \
        .withColumn("batting_avg_1000D", f.when(f.col("dismissals_1000D") > 0, f.col("batter_runs_1000D")/f.col("dismissals_1000D")).otherwise(f.col("batter_runs_1000D"))) \
        .withColumn("batting_avg_1000D_venue", f.when(f.col("dismissals_1000D_venue") > 0, f.col("batter_runs_1000D_venue")/f.col("dismissals_1000D_venue")).otherwise(f.col("batter_runs_1000D_venue")))

feature_df_with_bat_avg_sr = feature_df_with_bat_avg\
    .withColumn("batting_sr_30D", f.when(f.col("balls_faced_30D") > 0, f.col("batter_runs_30D")/f.col("balls_faced_30D")).otherwise(f.col("batter_runs_30D"))) \
    .withColumn("batting_sr_90D", f.when(f.col("balls_faced_90D") > 0, f.col("batter_runs_90D")/f.col("balls_faced_90D")).otherwise(f.col("batter_runs_90D"))) \
    .withColumn("batting_sr_300D", f.when(f.col("balls_faced_300D") > 0, f.col("batter_runs_300D")/f.col("balls_faced_300D")).otherwise(f.col("batter_runs_300D"))) \
    .withColumn("batting_sr_1000D", f.when(f.col("balls_faced_1000D") > 0, f.col("batter_runs_1000D")/f.col("balls_faced_1000D")).otherwise(f.col("batter_runs_1000D"))) \
    .withColumn("batting_sr_1000D_venue", f.when(f.col("balls_faced_1000D_venue") > 0, f.col("batter_runs_1000D_venue")/f.col("balls_faced_1000D_venue")).otherwise(f.col("batter_runs_1000D_venue")))

feature_df_with_bat_avg_sr_bowl_avg = feature_df_with_bat_avg_sr\
        .withColumn("bowling_avg_30D", f.when(f.col("wicket_sum_30D") > 0, f.col("total_runs_30D")/f.col("wicket_sum_30D")).otherwise(f.col("total_runs_30D"))) \
        .withColumn("bowling_avg_90D", f.when(f.col("wicket_sum_90D") > 0, f.col("total_runs_90D")/f.col("wicket_sum_90D")).otherwise(f.col("total_runs_90D"))) \
        .withColumn("bowling_avg_300D", f.when(f.col("wicket_sum_300D") > 0, f.col("total_runs_300D")/f.col("wicket_sum_300D")).otherwise(f.col("total_runs_300D"))) \
        .withColumn("bowling_avg_1000D", f.when(f.col("wicket_sum_1000D") > 0, f.col("total_runs_1000D")/f.col("wicket_sum_1000D")).otherwise(f.col("total_runs_1000D"))) \
        .withColumn("bowling_avg_1000D_venue", f.when(f.col("wicket_sum_1000D_venue") > 0, f.col("total_runs_1000D_venue")/f.col("wicket_sum_1000D_venue")).otherwise(f.col("total_runs_1000D_venue")))

feature_df_with_bat_avg_sr_bowl_avg_sr = feature_df_with_bat_avg_sr_bowl_avg\
    .withColumn("bowling_sr_30D", f.when(f.col("wicket_sum_30D") > 0, f.col("deliveries_30D")/f.col("wicket_sum_30D")).otherwise(f.col("deliveries_30D"))) \
    .withColumn("bowling_sr_90D", f.when(f.col("wicket_sum_90D") > 0, f.col("deliveries_90D")/f.col("wicket_sum_90D")).otherwise(f.col("deliveries_90D"))) \
    .withColumn("bowling_sr_300D", f.when(f.col("wicket_sum_300D") > 0, f.col("deliveries_300D")/f.col("wicket_sum_300D")).otherwise(f.col("deliveries_300D"))) \
    .withColumn("bowling_sr_1000D", f.when(f.col("wicket_sum_1000D") > 0, f.col("deliveries_1000D")/f.col("wicket_sum_1000D")).otherwise(f.col("deliveries_1000D"))) \
    .withColumn("bowling_sr_1000D_venue", f.when(f.col("wicket_sum_1000D_venue") > 0, f.col("deliveries_1000D_venue")/f.col("wicket_sum_1000D_venue")).otherwise(f.col("deliveries_1000D_venue")))

all_features_df = feature_df_with_bat_avg_sr_bowl_avg_sr\
    .withColumn("bowling_eco_30D", f.when(f.col("deliveries_30D") > 0, f.col("total_runs_30D") * 6.0/f.col("deliveries_30D")).otherwise(f.col("total_runs_30D") * 6.0)) \
    .withColumn("bowling_eco_90D", f.when(f.col("deliveries_90D") > 0, f.col("total_runs_90D") * 6.0/f.col("deliveries_90D")).otherwise(f.col("total_runs_90D") * 6.0)) \
    .withColumn("bowling_eco_300D", f.when(f.col("deliveries_300D") > 0, f.col("total_runs_300D") * 6.0/f.col("deliveries_300D")).otherwise(f.col("total_runs_300D") * 6.0)) \
    .withColumn("bowling_eco_1000D", f.when(f.col("deliveries_1000D") > 0, f.col("total_runs_1000D") * 6.0/f.col("deliveries_1000D")).otherwise(f.col("total_runs_1000D") * 6.0)) \
    .withColumn("bowling_eco_1000D_venue", f.when(f.col("deliveries_1000D_venue") > 0, f.col("total_runs_1000D_venue") * 6.0/f.col("deliveries_1000D_venue")).otherwise(f.col("total_runs_1000D_venue") * 6.0))



In [29]:
player_last_names = [
    "healy",
    "mooney",
    "jafta",
    "tucker",
    "lanning",
    "wolvaardt",
    "luus",
    "mcgrath",
    "bosch",
    "harris",
    "goodall",
    "brits",
    "dercksen",
    "macheke",
    "perry",
    "kapp",
    "gardner",
    "klerk",
    "tryon",
    "sutherland",
    "garth",
    "andrews",
    "schutt",
    "jonassen",
    "ismail",
    "klaas",
    "king",
    "brown",
    "khaka",
    "wareham",
    "mlaba",
    "graham",
    "sekhumkhune"
]

In [30]:
all_players_df = spark.read.csv("downloads/people.csv", header=True)
all_players_df.registerTempTable("all_players")
all_players_df.toPandas()

from string import Template
name_predicates = []
name_clause_template = Template("lower(reverse(split(unique_name,' '))[0]) = '$player_name'")
for name in player_last_names:
    name_predicates.append(name_clause_template.substitute(player_name=name))
name_clause = " OR ".join(name_predicates)
player_id_names_df = spark.sql("select identifier, unique_name from all_players where %s"%name_clause)

player_features_df = all_features_df.join(
    player_id_names_df,
    all_features_df.player_id ==  player_id_names_df.identifier,"inner" )
player_features_df.count()

48

In [31]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor ,GBTRegressionModel
from path_manager import model_train_input_path, model_test_input_path, model_train_predictions_path, model_test_predictions_path, model_save_artifact_path
from model_trainer import features

In [32]:
loaded_gbtr = GBTRegressionModel.load(model_save_artifact_path)

In [33]:
va = VectorAssembler(inputCols = features, outputCol='features')

In [34]:
va_inference_df = va.transform(player_features_df)

In [35]:
inference_predictions_df = loaded_gbtr.transform(va_inference_df)

In [36]:
inference_predictions_df.show()

+---------+--------------------+-------+-----------------+-----------------------+-----------------+-----------------------+----------------+----------------------+--------------------+--------------------------+---------------+---------------------+----------------+----------------------+----------------+---------------+-------------------+--------------+---------------+---------------------+---------------+--------------+------------------+-------------+---------------+---------------------+---------------+--------------+------------------+-------------+----------------+----------------------+----------------+----------------------+----------------+----------------------+------------------+------------------------+---------------+---------------+---------------+-----------------+--------------+--------------+--------------+----------------+--------------+--------------+--------------+----------------+-------------------------+-----------------------+---------------+------------------

In [37]:
inference_predictions_df.registerTempTable("player_inferences")

In [38]:
import pandas as pd
pd.set_option('display.max_rows', 500)

In [39]:
spark.sql("""
select player_id, unique_name, 
            prediction, rank() over(order by prediction desc) as predicted_rank
        from player_inferences
""").toPandas()

23/02/18 22:29:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,player_id,unique_name,prediction,predicted_rank
0,0b69a844,KJ Garth,10.232646,1
1,108c4c09,S Ismail,10.232646,1
2,99812d10,H Graham,10.232646,1
3,83558266,A King,10.232646,1
4,17608a6f,ML Schutt,10.232646,1
5,43223c50,GA Harris,8.660424,6
6,ce85fcaf,M Klaas,8.660424,6
7,77851c2d,SACA King,8.660424,6
8,98797fb4,K Sutherland,8.660424,6
9,34304210,S Jafta,8.660424,6


In [40]:
player_features_df.count()

48